# Image Degredation Testing

How robust are images classifiers to various different degredations in the image?

This notebook provides a set of functions for testing the ability of an image classifier to correctly classify images that have been systematically degraded in a number of different ways.





## Incremental Cumulative Degredations
These degredations can be applied repeatedly to incrementally degrade an image, removing the need to start from a fresh version of the image each time.

Some have a random component, so the results might be different each time they are run. Others are deterministic. This is noted in the docstring comment for each function.

None of the functions return an image, they all manipulate the given image in place

In [3]:
def rand_line(img, color):
    '''
    Called by rand_line_black or rand_line_white - don't call directly
    '''
    w = img.shape[1]
    h = img.shape[0]
    # Start point is random position on either x = 0 or y = 0
    if random.random() > 0.5:
        sx = 0
        sy = random.randint(0, h-1)
    else:
        sx = random.randint(0, w-1)
        sy = 0
    # End point is random position on either x = w or y = h
    if random.random() > 0.5:
        ex = w
        ey = random.randint(0, h-1)
    else:
        ex = random.randint(0, w-1)
        ey = h
    img = cv2.line(img, (sx, sy), (ex, ey), color, 1, lineType=cv2.LINE_AA)

def rand_line_black(img):
    '''
    Add a single black line to the image of 1 pixel thickness and at a random orientation. All lines start and end at an image edge
    (so 0 or width or height)
    
    Behaviour: Stochastic
    '''
    rand_line(img, (0, 0, 0))

def rand_line_white(img):
    '''
    Add a single white line to the image of 1 pixel thickness and at a random orientation. All lines start and end at an image edge
    (so 0 or width or height)
    
    Behaviour: Stochastic
    '''
    rand_line(img, (255, 255, 255))
    
    
def rand_occlude_boxes(img):
    w = img.shape[1]
    h = img.shape[0]
    numboxes = int((w+h)/10)
    for i in range(numboxes):
        ow = random.randint(2, 5)
        oh = random.randint(2, 5)
        ox = random.randint(0, w-ow)
        oy = random.randint(0, h-oh)
        cv2.rectangle(img, (ox, oy), (ox+ow, oy+oh), (0,0,0), -1)

def blur(img):
    '''
    Apply mean blurring with a 5 x 5 filter size
    
    Behaviour: Deterministic
    '''
    cv2.blur(img, (5, 5), dst=img)
    
def blur_rand_box(img):
    w = img.shape[1]
    h = img.shape[0]
    numboxes = int((w+h))
    for i in range(numboxes):
        ow = random.randint(2, 10)
        oh = random.randint(2, 10)
        ox = random.randint(0, w-ow)
        oy = random.randint(0, h-oh)
        box = img[ox:ox+ow, oy:oy+oh]
        cv2.blur(box, (5, 5), dst=box)
        img[ox:ox+ow, oy:oy+oh] = box
    
def speckle(img):
    '''
    Add a number of random coloured dots to the image
    
    Behaviour: Stochastic
    '''
    w = img.shape[1]
    h = img.shape[0]
    dots = int(w*h/50)
    for i in range(dots):
        y = random.randint(0, h-1)
        x = random.randint(0, w-1)
        img[y,x] = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

def fog(img):
    '''
    Add a number of random coloured dots to the image
    
    Behaviour: Stochastic
    '''
    w = img.shape[1]
    h = img.shape[0]
    dots = int(w*h/5)
    for i in range(dots):
        y = random.randint(0, h-1)
        x = random.randint(0, w-1)
        for j in range(3):
            img[y,x,j] = np.clip(20 + 1 * img[y,x,j], 0, 255)

def fade_to_black(img):
    cv2.convertScaleAbs(img, alpha=0.9, beta=0, dst=img)
    
def fade_to_white(img):
    cv2.convertScaleAbs(img, alpha=1.1, beta=0, dst=img)
    
def fade_to_gray(img):
    imageHSV = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    newSaturationChannel = 0.9 * imageHSV[:,:,1]
    imageHSV[:,:,1] = newSaturationChannel
    cv2.cvtColor(imageHSV, cv2.COLOR_HSV2RGB, dst=img)
    
def swap_rand_adj_pixels(img):
    '''
    Swap a number of random adjacent pixel pairs
    
    Behaviour: Stochastic
    '''
    w = img.shape[1]-1
    h = img.shape[0]-1
    num_pixels = int(w*h/2)
    for i in range(num_pixels):
        x1 = random.randint(0, w-1)
        y1 = random.randint(0, h-1)
        while True:
            x2 = x1+random.randint(-1, 1)
            y2 = y1+random.randint(-1, 1)
            if 0 <= x2 <= w and 0 <= y2 <= h:
                break
        p1 = img[y1][x1]
        img[y1][x1] = img[y2][x2]
        img[y2][x2] = p1
        
def swap_rand_pixels(img):
    w = img.shape[1]-1
    h = img.shape[0]-1
    num_pixels = int(w*h/20)
    for i in range(num_pixels):
        x1 = random.randint(0, w)
        y1 = random.randint(0, h)
        x2 = random.randint(0, w)
        y2 = random.randint(0, h)
        p1 = img[y1][x1]
        img[y1][x1] = img[y2][x2]
        img[y2][x2] = p1

# posterize
def posterise(img, levels):
    for i in range(levels):
        img[(img >= i*255/levels) & (img < (i+1)*255/levels)] = i*255/(levels-1)


# jpeg compress
def jpeg_compress(img, qual):
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), qual]
    result, encimg = cv2.imencode('.jpg', img, encode_param)
    de = cv2.imdecode(encimg,cv2.IMREAD_UNCHANGED)
    w = img.shape[1]
    h = img.shape[0]
    for i in range(w):
        for j in range(h):
            img[i,j] = de[i,j]


def degrade_img(img, degrade_fn):
    degrade_fn(img)

## Functions to Run the Experiments

In [ ]:
import math

def count_correct_preds(model, imgs, labels):
    imgs = preprocess_input(imgs)
    with tf.device('/GPU:1'):
        preds = model.predict(imgs)
    preds = decode_predictions(preds, top=1)
    correct = 0
    for i, pred in enumerate(preds):
        if pred[0][0] == labels[i][0]:
            correct += 1
    return correct / len(imgs)


def find_label(preds, label):
    for i in range(len(preds)):
        if preds[i][0] == label:
            return i
    return None

def average_rank_of_target_class(model, imgs, labels):
    '''Calculate the average rank in the ordered list of predictions of the target label across
    a dataset of images'''
    imgs = preprocess_input(imgs)
    with tf.device('/GPU:1'):
        preds = model.predict(imgs)
    preds = decode_predictions(preds, top=1000)
    #print(preds[0])
    avloc = 0
    avscore = 0
    for i, pred in enumerate(preds):
        #print("Looking for",labels[i][0])
        loc = find_label(pred, labels[i][0])
        #print(loc)
        avloc += loc
        if math.isnan(pred[0][2]):
            avscore += 0
            #print(pred[0])
        else:
            avscore += pred[0][2]
    return (avloc / len(imgs), avscore / len(imgs))

def incremental_dataset_degrade(model, imgs, labels, degrade_fn, maxiters = 30):
    '''
    Repeatedly apply one image degrade function to all the images in a folder and calculate the classification
    accuracy of the degraded dataset. Only works on degradation functions that are cumulative.
    
    Arguments:
    imgs: Array of images to process
    labels: Correct labels for the images
    degrade_fn: A function to call to degrade the images
    maxiters = 20: Maximum number of iterations
    '''
    
    accuracies = []
    label_ranks = []
    # Baseline before degrading
    accuracies.append(count_correct_preds(model, imgs, labels))
    label_ranks.append(average_rank_of_target_class(model, imgs, labels))
    for i in range(maxiters):
        for j, img in enumerate(imgs):
            img = degrade_img(img, degrade_fn)
        accuracies.append(count_correct_preds(model, imgs, labels))
        label_ranks.append(average_rank_of_target_class(model, imgs, labels))
        print(accuracies[-1], label_ranks[-1])  
    return label_ranks, accuracies

